# 第1章：基本提示结构

- [课程](#lesson)
- [练习](#exercises)
- [示例练习场](#example-playground)

## 设置

运行以下设置单元格来加载您的API密钥并建立`get_completion`辅助函数。

In [ ]:
%pip install anthropic --quiet

# Import the hints module from the utils package
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import hints

# Import python's built-in regular expression library
import re
from anthropic import AnthropicBedrock

%store -r MODEL_NAME
%store -r AWS_REGION

client = AnthropicBedrock(aws_region=AWS_REGION)

def get_completion(prompt, system=''):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": prompt}
        ],
        system=system
    )
    return message.content[0].text

---

## 课程

Anthropic提供两个API，遗留的[文本补全API](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-text-completion.html)和当前的[消息API](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-messages.html)。在本教程中，我们将专门使用消息API。

至少，使用消息API调用Claude需要以下参数：
- `model`: 您打算调用的模型的[API模型名称](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html#model-ids-arns)

- `max_tokens`: 停止前生成的最大令牌数。请注意，Claude可能在达到此最大值之前停止。此参数仅指定要生成的令牌的绝对最大数量。此外，这是一个*硬*停止，意味着它可能导致Claude在单词或句子中间停止生成。

- `messages`: 输入消息数组。我们的模型经过训练，可以在交替的`user`和`assistant`对话轮次上运行。创建新的`Message`时，您使用messages参数指定先前的对话轮次，然后模型生成对话中的下一个`Message`。
  - 每个输入消息必须是具有`role`和`content`的对象。您可以指定单个`user`角色消息，或者可以包含多个`user`和`assistant`消息（如果是这样，它们必须交替）。第一条消息必须始终使用user`role`。

还有可选参数，例如：
- `system`: 系统提示 - 下面会详细介绍。
  
- `temperature`: Claude响应中的可变性程度。对于这些课程和练习，我们将`temperature`设置为0。

有关所有API参数的完整列表，请访问我们的[API文档](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-claude.html)。

### 示例

让我们看看Claude如何响应一些格式正确的提示。对于以下每个单元格，运行单元格（`shift+enter`），Claude的响应将出现在块下方。

In [ ]:
# Prompt
PROMPT = "Hi Claude, how are you?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Print Claude's response
print(get_completion(PROMPT))

现在让我们看一些不包含正确消息API格式的提示。对于这些格式错误的提示，消息API会返回错误。

首先，我们有一个消息API调用的示例，该调用在`messages`数组中缺少`role`和`content`字段。

> ⚠️ **警告：** 由于提示中messages参数的格式不正确，以下单元格将返回错误。这是预期的行为。

In [ ]:
# Get Claude's response
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"Hi Claude, how are you?"}
        ]
    )

# Print Claude's response
print(response[0].text)

这是一个无法在`user`和`assistant`角色之间交替的提示。

> ⚠️ **警告：** 由于缺乏`user`和`assistant`角色之间的交替，Claude将返回错误消息。这是预期的行为。

In [ ]:
# Get Claude's response
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": "What year was Celine Dion born in?"},
          {"role": "user", "content": "Also, can you tell me some other facts about her?"}
        ]
    )

# Print Claude's response
print(response[0].text)

`user`和`assistant`消息**必须交替**，消息**必须以`user`轮次开始**。您可以在提示中有多个`user`和`assistant`对（就像模拟多轮对话一样）。您还可以在终端`assistant`消息中放入单词，让Claude从您停下的地方继续（更多内容将在后面的章节中介绍）。

#### 系统提示

您还可以使用**系统提示**。系统提示是在"User"轮次中向Claude提出问题或任务之前，**为Claude提供上下文、指令和指导原则**的一种方式。

在结构上，系统提示独立于`user`和`assistant`消息列表之外，因此属于单独的`system`参数（查看笔记本[设置](#setup)部分中`get_completion`辅助函数的结构）。

在本教程中，无论我们可能在哪里使用系统提示，我们都为您在完成函数中提供了一个`system`字段。如果您不想使用系统提示，只需将`SYSTEM_PROMPT`变量设置为空字符串即可。

#### 系统提示示例

In [ ]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

为什么要使用系统提示？**精心编写的系统提示可以在多个方面提高Claude的性能**，例如增强Claude遵循规则和指令的能力。有关更多信息，请访问我们关于[如何在Claude中使用系统提示](https://docs.anthropic.com/claude/docs/how-to-use-system-prompts)的文档。

现在我们将深入一些练习。如果您想在不更改上述任何内容的情况下试验课程提示，请滚动到课程笔记本的最底部访问[**示例练习场**](#example-playground)。

---

## 练习
- [练习 1.1 - 数到三](#exercise-11---counting-to-three)
- [练习 1.2 - 系统提示](#exercise-12---system-prompt)

### 练习 1.1 - 数到三
使用正确的`user`/`assistant`格式，编辑下面的`PROMPT`让Claude**数到三**。输出还将显示您的解决方案是否正确。

In [ ]:
# Prompt - this is the only field you should change
PROMPT = "[Replace this text]"

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    pattern = re.compile(r'^(?=.*1)(?=.*2)(?=.*3).*$', re.DOTALL)
    return bool(pattern.match(text))

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ 如果您需要提示，请运行下面的单元格！

In [ ]:
print(hints.exercise_1_1_hint)

### 练习 1.2 - 系统提示

修改`SYSTEM_PROMPT`让Claude像3岁小孩一样回应。

In [ ]:
# System prompt - this is the only field you should change
SYSTEM_PROMPT = "[Replace this text]"

# Prompt
PROMPT = "How big is the sky?"

# Get Claude's response
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search(r"giggles", text) or re.search(r"soo", text))

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ 如果您需要提示，请运行下面的单元格！

In [ ]:
print(hints.exercise_1_2_hint)

### 恭喜！

如果您已经解决了到目前为止的所有练习，那么您已经准备好进入下一章了。祝您提示愉快！

---

## 示例练习场

这是一个供您自由试验本课中展示的提示示例的区域，您可以调整提示来看看它如何影响Claude的回应。

In [ ]:
# Prompt
PROMPT = "Hi Claude, how are you?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Get Claude's response
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"Hi Claude, how are you?"}
        ]
    )

# Print Claude's response
print(response[0].text)

In [ ]:
# Get Claude's response
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": "What year was Celine Dion born in?"},
          {"role": "user", "content": "Also, can you tell me some other facts about her?"}
        ]
    )

# Print Claude's response
print(response[0].text)

In [ ]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))